In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [ ]:
alzheimer = pd.read_csv('/content/oasis_longitudinal.csv')
alzheimer_Modified = alzheimer.drop("Subject ID", axis = 1)
alzheimer_Modified = alzheimer_Modified.drop("MRI ID", axis = 1)
alzheimer_Modified.dropna(inplace=True)
alzheimer_Modified

In [ ]:
'''for i, v in enumerate(alzheimer.Group):
  if v == 'Converted':
    print(i, v)
    alzheimer.Group[i] = 'Demented'

alzheimer_Modified'''

In [ ]:
pd.crosstab(alzheimer_Modified.CDR, alzheimer_Modified.Group)

In [ ]:
#alzheimer_Modified.describe()

In [ ]:
#alzheimer_Modified["MMSE"].value_counts()

In [ ]:
#alzheimer_Modified["SES"].value_counts()

In [ ]:
np.random.seed(42)
x = alzheimer_Modified.drop("Group", axis = 1)
y = alzheimer_Modified["Group"]

In [ ]:
features = ['M/F', 'Hand']
one_hot = OneHotEncoder()
transform = ColumnTransformer([('one_hot', one_hot, features)], remainder='passthrough')
transfromed_X = transform.fit_transform(x)
pd.DataFrame(transfromed_X)
#df.rename('Female', 'Male', 'Hand', 'Visit', 'Age', 'Mr Delay', 'Age', 'Education', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF')

In [ ]:
'''
corr_matrix = alzheimer_Modified.corr()
fig, ax = plt.subplots(figsize = (15, 10))
ax = sns.heatmap(corr_matrix,
                 annot = True,
                 linewidths = .5,
                 fmt = ".3f",
                 cmap = "YlGnBu")
'''

In [ ]:
'''corr_matrix = pd.DataFrame(transfromed_X).corr()
fig, ax = plt.subplots(figsize = (15, 10))
ax = sns.heatmap(corr_matrix,
                 annot = True,
                 linewidths = .5,
                 fmt = ".3f",
                 cmap = "YlGnBu")'''

In [ ]:
def evaluation_pred(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred,
                              pos_label='positive',
                              average='micro')
  recall = recall_score(y_true, y_pred,
                        pos_label='positive',
                        average='micro')
  f1 = f1_score(y_true, y_pred,
                        pos_label='positive',
                        average='micro')
  martic_dict = {"Accuracy": round(accuracy, 2),
                 "precision": round(precision, 2),
                 "recall": round(recall, 2),
                 "f1": round(f1, 2)}

  print(f"accuracy: {accuracy * 100:.2f}%")
  print(f"percision {precision:.2f}")
  print(f"recall {recall:.2f}")
  print(f"f1: {f1:.02f}")

  return martic_dict

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def plot_roc_curve(fpr, tpr, name, val):
  plt.plot(fpr, tpr, color = "orange", label="AUC ({})".format(val))
  plt.plot([0,1],[0,1], color = "darkblue",linestyle="--",label="Guessing")
  plt.xlabel("False possitive rate")
  plt.ylabel("True possitive rate")
  plt.title("ROC Curve (With MRI data)" + name)
  plt.legend()
  plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(transfromed_X, y, test_size=0.2)

In [ ]:
#import pickle

#pickle.dump(rs_clf, open("RandomForestCV", "wb"))

In [ ]:
parameters = {
    'priors': [None],
    'var_smoothing': [0.000000001, 0.0000001, 0.00001]
}

cv_gnb = GridSearchCV(estimator=GaussianNB(), param_grid=parameters, cv = 5, verbose=2  )
cv_gnb.fit(x_train, y_train)
y_pred_gnb = cv_gnb.predict(x_test)
evaluation_pred(y_test, y_pred_gnb)

In [ ]:
cv_gnb.best_params_

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

param_grid = {'C': [10, 200, 400, 600], 
              'dual': [False, True],
              'loss': ['squared_hinge','hinge'],
              'tol': [.00001, .0001, .001],
              'multi_class':['ovr']} 

cv_svm = GridSearchCV(estimator=LinearSVC(), param_grid=param_grid, cv = 5, verbose=2)
cv_svm.fit(x_train, y_train)
y_pred_gnb = cv_svm.predict(x_test)
evaluation_pred(y_test, y_pred_gnb)

In [ ]:
cv_svm.best_params_

In [ ]:
param_grid = {'C': [100,275,500],
              'penalty': ['l2'], 
              'tol': [.000001],
              'fit_intercept':[True],
              'class_weight':['balanced'],
              'solver':['liblinear']
              }  
cv_lr = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv = 5, verbose=2)
cv_lr.fit(x_train, y_train)
y_pred_lr = cv_lr.predict(x_test)
evaluation_pred(y_test, y_pred_lr)

In [ ]:
cv_lr.best_params_

In [ ]:
def plot_conf_mat(y_test, y_pred_lr):
  fig, ax = plt.subplots(figsize=(3,3))
  ax = sns.heatmap(confusion_matrix(y_test, y_pred_lr),
                   annot=True,
                   cbar=False)
  plt.xlabel("True Label")
  plt.ylabel("Predicted label")
  

In [ ]:
plot_conf_mat(y_test, y_pred_lr)

In [ ]:
cv_ad.best_params_

In [ ]:
turfc = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                                                criterion='gini', max_depth=10, max_features='auto',
                                                max_leaf_nodes=None, max_samples=None,
                                                min_impurity_decrease=0.0, min_impurity_split=None,
                                                min_samples_leaf=1, min_samples_split=4,
                                                min_weight_fraction_leaf=0.0, n_estimators=10,
                                                n_jobs=None, oob_score=False, random_state=None,
                                                verbose=0, warm_start=False)

turfc.fit(x_train, y_train)
y_pred_lr = turfc.predict(x_test)
evaluation_pred(y_test, y_pred_lr)

In [ ]:
param_grid = {'n_neighbors': [10],
              'weights': ['distance'],
              'algorithm':['auto'],
              'leaf_size': [10],
              'p':[1,2],
              'metric':['manhattan'],
              'n_jobs':[-1]}

cv_kn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, cv = 5, verbose=2)
cv_kn.fit(x_train, y_train)
y_pred_lr = cv_kn.predict(x_test)
evaluation_pred(y_test, y_pred_lr)

In [ ]:
cv_kn.best_params_

In [ ]:
grid = {
    "n_estimators": [20, 200, 500],
    "max_depth": [None, 3, 10, 20],
    "max_features": ["auto"],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [12, 5, 10],
    'criterion':['gini'],
}
clf = RandomForestClassifier(n_jobs=1)

rs_clf = GridSearchCV(estimator=clf,
                      param_grid = grid,
                      cv = 10,
                      verbose = 2)

rs_clf.fit(x_train, y_train)
rs_clf.fit(x_train, y_train)
y_pred_lr = rs_clf.predict(x_test)
evaluation_pred(y_test, y_pred_lr)

In [ ]:
rs_clf.best_params_

In [ ]:
rs_clf = RandomForestClassifier( max_depth = 10,
                              max_features= 'auto',
                              min_samples_leaf= 5,
                              min_samples_split= 10,
                              n_estimators= 10,
                              criterion='gini')

gnb = GaussianNB(priors= None, var_smoothing= 1e-08)
cv_svm = LinearSVC(C= 200,
                dual= False,
                loss= 'squared_hinge',
                multi_class= 'ovr',
                tol= 1e-05)
cv_lr= LogisticRegression(C= 225,
                      class_weight= 'balanced',
                      fit_intercept= True,
                      penalty= 'l2',
                      solver= 'liblinear',
                      tol= 1e-06)
cv_kn = KNeighborsClassifier()
Ada = AdaBoostClassifier()

param_grid = {'base_estimator':[rs_clf,cv_lr,cv_svm],
              'n_estimators': [100, 700, 1200],
              'algorithm':['SAMME']
              } 
cv_ad = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=param_grid, cv = 5, verbose=2)
cv_ad.fit(x_train, y_train)
y_pred_lr = cv_ad.predict(x_test)
evaluation_pred(y_test, y_pred_lr)

confusion_matrix(y_test, y_pred_lr)

In [ ]:
cv_ad.best_params_

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(),
    "GaussianNB": GaussianNB(),
    "Linear SVC": LinearSVC(max_iter=1000000, dual = True),
    "K Neighbours Classifier": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Ada Boost": AdaBoostClassifier()
}

def fitAndScore(models, x_train, x_test, y_train, y_test):
  np.random.seed(42)
  scores = {}
  for name, model in models.items():
    model.fit(x_train, y_train)
    scores[name] = model.score(x_test, y_test)
    print(name)
    y_preds = model.predict(x_test)
    evaluation_pred(y_test, y_preds)
  return scores

model_scores = fitAndScore(models=models,x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)
model_scores

In [ ]:
models = {
    "Random Forest": rs_clf,
    "GaussianNB": cv_gnb,
    "Linear SVC": cv_svm,
    "K Neighbours Classifier": cv_kn,
    "Logistic Regression": cv_lr,
    "Ada Boost": cv_ad
}
def fitAndScore(models, x_train, x_test, y_train, y_test):
  np.random.seed(42)
  scores = {}
  for name, model in models.items():
    model.fit(x_train, y_train)
    scores[name] = model.score(x_test, y_test)
    print(name)
    y_preds = model.predict(x_test)
    evaluation_pred(y_test, y_preds)
  return scores

model_scores = fitAndScore(models=models,x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)
model_scores

In [ ]:
from sklearn.feature_selection import SelectFromModel

clf = rs_clf
gnb = gnb
lsvc = cv_svm
lg =  cv_lr
adb =  cv_ad

In [ ]:
clf.fit(x_train, y_train)
y_prob = clf.predict_proba(x_test)
y_prob_pos = y_prob[:, 1]
fpr, tpr, threshold = roc_curve(y_test, y_prob_pos, pos_label='Demented')
val = roc_auc_score(y_test, y_prob, multi_class='ovr')
plot_roc_curve(fpr,tpr, 'Random Forest Classifier', val)

In [ ]:
gnb.fit(x_train, y_train)
y_prob = gnb.predict_proba(x_test)
y_prob_pos = y_prob[:, 1]
fpr, tpr, threshold = roc_curve(y_test, y_prob_pos, pos_label='Demented')
val = roc_auc_score(y_test, y_prob, multi_class='ovr')
plot_roc_curve(fpr,tpr, 'GaussianNB', val)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(transfromed_X, y, test_size=0.2)
lg.fit(x_train, y_train)
y_prob = lg.predict_proba(x_test)
y_prob_pos = y_prob[:, 1]
fpr, tpr, threshold = roc_curve(y_test, y_prob_pos, pos_label='Demented')
val = roc_auc_score(y_test, y_prob, multi_class='ovr')
plot_roc_curve(fpr,tpr, 'Logistic Regression', val)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(transfromed_X, y, test_size=0.2)
adb.fit(x_train, y_train)
y_prob = adb.predict_proba(x_test)
y_prob_pos = y_prob[:, 1]
fpr, tpr, threshold = roc_curve(y_test, y_prob_pos, pos_label='Demented')
val = roc_auc_score(y_test, y_prob, multi_class='ovr')
plot_roc_curve(fpr,tpr, 'AdaBoost Classifier', val)

In [ ]:
cclf = CalibratedClassifierCV(base_estimator=LinearSVC(C= 500,
                dual= False,
                loss= 'squared_hinge',
                multi_class= 'ovr',
                tol= 1e-08), cv=5)
cclf.fit(x_train, y_train)
y_prob = cclf.predict_proba(x_test)
y_prob_pos = y_prob[:, 1]
fpr, tpr, threshold = roc_curve(y_test, y_prob_pos, pos_label='Demented')
val = roc_auc_score(y_test, y_prob, multi_class='ovr')
plot_roc_curve(fpr,tpr, 'Linear SVC', val)

In [ ]:
#fr_df = pd.DataFrame(features_dict, index=[0])
#alzheimer_Modified.T.plot.bar(title="Importance", legend= False)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

model = VotingClassifier(estimators=[('clf', clf), ('adb', Ada), ('lg', LG), ('svc', cclf)], voting='soft')
model.fit(x_train,y_train)
hybrid_score = model.score(x_test,y_test)
m_pred_y = model.predict(x_test)
evaluation_pred(y_test, m_pred_y)
hybrid_score

In [ ]:
model.fit(x_train, y_train)
y_prob = model.predict_proba(x_test)
y_prob_pos = y_prob[:, 1]
fpr, tpr, threshold = roc_curve(y_test, y_prob_pos, pos_label='Demented')
val = roc_auc_score(y_test, y_prob, multi_class='ovr')
plot_roc_curve(fpr,tpr, 'Hybrid', val)

In [ ]:
model_scores['hybrid'] = hybrid_score
model_compare = pd.DataFrame(model_scores, index=['accuracy'])
model_compare.T.plot(kind='barh')